# Lab: PBDW data assimilation on a 1D diffusion model

**Goal.** Implement and test the core pieces of the *Parametrized-Background Data-Weak (PBDW)* method:
- build a reduced **background space** $\mathcal{Z}_N$ from snapshots (POD),
- build an **observable space** $\mathcal{U}_M$ from sensors (Riesz representers),
- implement **offline** (assemble the saddle-point system) and **online** (reconstruct from data),
- study the effects of **noise**, **number of sensors**, and **sensor placement** (GEIM).

**How to use this notebook.** Work top-to-bottom. Cells marked **TODO** must be completed for the notebook to run.

We use a uniform grid and the $V=L^2(0,1)$ inner product:
$$
(u,v)_V \approx \Delta x\,u^Tv.
$$
Sensors are weighted averages $y_m=\int w_m u$; discretely
`observe(u, W, dx) = dx*(W @ u)`.

A complete reference implementation is provided in the companion notebook `PBDW_DataAssimilation_fixed.ipynb`.


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.linalg import solve, lstsq
from scipy.sparse import diags
from scipy.sparse.linalg import spsolve
from dataclasses import dataclass
from typing import Tuple, List, Callable

np.random.seed(42)
plt.rcParams['figure.figsize'] = (12, 4)

## 1. Problem Setup: 1D Diffusion Equation

We consider:
$$-\nabla \cdot (\kappa(x;\mu) \nabla u) = f(x), \quad x \in [0,1]$$
with $u(0) = u(1) = 0$ and parametrized conductivity.

In [ ]:
# Domain discretization
Nq = 200
x = np.linspace(0, 1, Nq)
dx = x[1] - x[0]

# Parameter space: mu in [0.5, 2.0]^3 for conductivity variations
P = 3  # Number of parameters
mu_min = np.array([0.5, 0.5, 0.5])
mu_max = np.array([2.0, 2.0, 2.0])

def build_conductivity(mu):
    """Build conductivity field κ(x;μ) = 1 + sum_i (μ_i - 1) * φ_i(x)"""
    centers = np.array([0.25, 0.5, 0.75])
    width = 0.2
    kappa = np.ones(Nq)
    for i in range(P):
        phi_i = np.exp(-((x - centers[i])**2) / (2 * width**2))
        kappa += (mu[i] - 1) * phi_i
    return kappa

def solve_diffusion(kappa):
    """Solve -∇·(κ∇u) = f with finite differences"""
    f = 10 * np.sin(2 * np.pi * x)  # Source term
    
    # Stiffness matrix
    kappa_half = 0.5 * (kappa[:-1] + kappa[1:])
    N_int = Nq - 2
    
    diag = (kappa_half[1:N_int+1] + kappa_half[0:N_int]) / dx**2
    upper = -kappa_half[1:N_int] / dx**2
    lower = -kappa_half[1:N_int] / dx**2
    
    A = diags([lower, diag, upper], [-1, 0, 1], format='csr')
    u_int = spsolve(A, f[1:-1])
    
    u = np.zeros(Nq)
    u[1:-1] = u_int
    return u

def sample_parameters(n):
    """Sample n parameters uniformly in parameter space"""
    return mu_min + np.random.rand(n, P) * (mu_max - mu_min)

# Test
mu_test = np.array([1.5, 0.8, 1.2])
kappa_test = build_conductivity(mu_test)
u_test = solve_diffusion(kappa_test)

fig, axes = plt.subplots(1, 2, figsize=(12, 3))
axes[0].plot(x, kappa_test, 'b-', linewidth=2)
axes[0].set_xlabel('x'); axes[0].set_ylabel('κ(x)')
axes[0].set_title(f'Conductivity (μ={mu_test})')
axes[0].grid(True, alpha=0.3)

axes[1].plot(x, u_test, 'r-', linewidth=2)
axes[1].set_xlabel('x'); axes[1].set_ylabel('u(x)')
axes[1].set_title('Solution')
axes[1].grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

## 2. Background space $\mathcal{Z}_N$ via POD

We generate a *training set* of solutions $\{u(\mu^i)\}_{i=1}^{n_{\mathrm{train}}}$ of the parametrized PDE
and compute a Proper Orthogonal Decomposition (POD) basis.

Let `S` be the snapshot matrix whose columns are the training solutions. The POD basis is obtained from an SVD
`S = U Σ Vᵀ`; we set
$$
\mathcal{Z}_N = \mathrm{span}\{U_1,\dots,U_N\}.
$$

This space represents what the model can already explain (anticipated physics).


**TODO (implementation).** In the code cell below:
1. Build the snapshot matrix `S`.
2. Compute an SVD and select a POD basis `Z`.
3. Plot singular values and the cumulative energy.


In [ ]:
# Generate training snapshots
n_train = 100
mu_train = sample_parameters(n_train)

# TODO 2.1: Build the snapshot matrix S (shape: Nq × n_train).
# Each column i should be u(mu_train[i]).
S = np.zeros((Nq, n_train))
for i, mu in enumerate(mu_train):
    # kappa = ...
    # S[:, i] = ...
    raise NotImplementedError("TODO 2.1: fill snapshot generation loop")

# TODO 2.2: POD via SVD: S = U Σ Vᵀ
U_pod, sigma, Vt = np.linalg.svd(S, full_matrices=False)

# TODO 2.3: Choose N and define the background basis Z = U[:, :N]
N = 8
Z = U_pod[:, :N]

# TODO 2.4: Compute and plot the cumulative POD energy
energy = np.cumsum(sigma**2) / np.sum(sigma**2)

fig, axes = plt.subplots(1, 2, figsize=(12, 4))
axes[0].semilogy(sigma[:30], 'bo-')
axes[0].axvline(N, color='r', linestyle='--', label=f'N={N}')
axes[0].set_xlabel('Mode'); axes[0].set_ylabel('Singular value')
axes[0].set_title('POD singular values')
axes[0].legend(); axes[0].grid(True, alpha=0.3)

axes[1].plot(energy[:30], 'go-')
axes[1].axvline(N, color='r', linestyle='--', label=f'N={N}: {energy[N-1]:.2%}')
axes[1].axhline(0.99, color='gray', linestyle=':')
axes[1].set_xlabel('Modes'); axes[1].set_ylabel('Cumulative energy')
axes[1].set_title('POD energy')
axes[1].legend(); axes[1].grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

print(f"Background space Z_N: N={N} modes, {energy[N-1]:.2%} energy captured")


## 3. Sensors and observable space $\mathcal{U}_M$

A *sensor* is a continuous linear functional $\ell_m: V \to \mathbb{R}$.
In this notebook we use **localized averages**
$$
\ell_m(v) = \int_0^1 w_m(x)\,v(x)\,dx,
$$
where $w_m$ is a normalized bump (Gaussian/box). With the $L^2$ inner product, the **Riesz representer**
satisfying $(q_m,v)_V = \ell_m(v)$ is simply $q_m = w_m$. Therefore
$$
\mathcal{U}_M = \mathrm{span}\{q_1,\dots,q_M\}.
$$

Discretization (uniform grid):
- `W` is the matrix of sensor *densities* $w_m(x_i)$ (rows).
- Measurements are computed by quadrature: `y = observe(u, W, dx) = dx*(W @ u)`.
- The matrix of Riesz representers is `Q = W.T` (columns $q_m$).


**TODO (implementation).** In the code cell below:
1. Implement `build_sensor_matrix` for `dirac`, `gaussian`, and `box` sensors.
2. Implement `observe(u, W, dx)`.
3. Implement `build_riesz_representers(W, dx)` for the L2 inner product.

*Hint:* define sensors as **densities** `w_m` with `dx*np.sum(w_m)=1`. Then `observe(u,W,dx)=dx*(W@u)` approximates $\int w_m u$. For `dirac`, choose `w_m = e_i/dx` so the observation is approximately `u(x_i)`.


In [ ]:
@dataclass
class SensorConfig:
    """Configuration for sensor setup"""
    positions: np.ndarray  # Sensor center positions
    width: float           # Sensor width (for Gaussian/box)
    type: str              # 'dirac', 'gaussian', 'box'

def build_sensor_matrix(x: np.ndarray, config: SensorConfig) -> np.ndarray:
    """
    Build sensor density matrix W (M × Nq), where row m approximates w_m(x_i).

    Convention used throughout the notebook:
      y = observe(u, W, dx) = dx*(W @ u) ≈ ∫ w_m(x) u(x) dx.
    """
    M = len(config.positions)
    Nq = len(x)
    dx = x[1] - x[0]
    W = np.zeros((M, Nq))

    # TODO 3.1: fill W row-by-row depending on config.type.
    # - 'dirac': choose the closest grid index i and set W[m, i] = 1/dx
    # - 'gaussian': set W[m, :] proportional to exp(-(x-pos)^2/(2*width^2)) and normalize so dx*sum = 1
    # - 'box': set W[m, mask]=const on |x-pos|<=width/2 and normalize so dx*sum = 1
    raise NotImplementedError("TODO 3.1: implement build_sensor_matrix")

def observe(u: np.ndarray, W: np.ndarray, dx: float) -> np.ndarray:
    """Return measurements y_m = ∫ w_m(x) u(x) dx via quadrature."""
    # TODO 3.2
    raise NotImplementedError("TODO 3.2: implement observe")

def build_riesz_representers(W: np.ndarray, dx: float) -> np.ndarray:
    """
    Return Q (Nq × M) whose columns are the Riesz representers q_m in V.

    For V=L2 and ell_m(v)=∫ w_m v, we have q_m = w_m.
    """
    # TODO 3.3
    raise NotImplementedError("TODO 3.3: implement build_riesz_representers")

# Configure sensors (you may change M, type, width later)
M = 15
sensor_positions = np.linspace(0.05, 0.95, M)
sensor_config = SensorConfig(positions=sensor_positions, width=0.05, type='gaussian')

# Build sensor matrix and Riesz representers
W = build_sensor_matrix(x, sensor_config)
Q = build_riesz_representers(W, dx)

print(f"Sensor matrix W: {W.shape}")
print(f"Observable space Q (Riesz representers): {Q.shape}")

# Visualize a few sensors (plot quadrature weights w_m * dx)
fig, ax = plt.subplots(figsize=(12, 4))
for m in range(0, M, 3):
    ax.plot(x, W[m, :] * dx, label=f'Sensor {m+1}' if m < 6 else None)
ax.scatter(sensor_positions, np.zeros(M), c='red', s=50, zorder=5, label='Sensor centers')
ax.set_xlabel('x'); ax.set_ylabel('quadrature weight')
ax.set_title(f'{sensor_config.type.capitalize()} sensors (M={M})')
ax.legend(); ax.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()


## 4. PBDW formulation (saddle-point system)

We seek an estimator of the form
$$
u_{N,M} = z_N + \eta_M, \qquad z_N \in \mathcal{Z}_N,\ \eta_M \in \mathcal{U}_M,
$$
where $\mathcal{Z}_N$ is a reduced **background/model** space (POD) and
$\mathcal{U}_M$ is the **observable/update** space spanned by the Riesz representers $\{q_m\}_{m=1}^M$.

With $\ell_m(v) = (q_m, v)_V$ and data $y_m^{\mathrm{obs}} = \ell_m(u^{\mathrm{true}})$, the (noise-free) PBDW conditions are:

- **Data matching:** $(q_m, u_{N,M})_V = y_m^{\mathrm{obs}}$, for $m=1,\dots,M$.
- **Minimal update:** $(\eta_M, z)_V = 0$ for all $z \in \mathcal{Z}_N$ (selects the smallest correction in the $V$-norm).

In coefficient form, with bases $Z=[z_1,\dots,z_N]$ and $Q=[q_1,\dots,q_M]$,
this yields the saddle-point linear system
$$
\begin{pmatrix}
A & B \\
B^T & 0
\end{pmatrix}
\begin{pmatrix}
\eta \\ z
\end{pmatrix}
=
\begin{pmatrix}
y^{\mathrm{obs}} \\ 0
\end{pmatrix},
\qquad
A = (Q,Q)_V,\ \ B = (Q,Z)_V.
$$

The *inf-sup constant* $\beta_{N,M}$ (computed below) controls stability and appears in a standard bound
$\|u^{\mathrm{true}}-u_{N,M}\|_V \le (1+1/\beta_{N,M})\,\inf_{z\in\mathcal{Z}_N}\|u^{\mathrm{true}}-z\|_V$ in the idealized setting.


**TODO (implementation).** Implement the offline and online routines in the next code cell. Your implementation should:
- build A and B using the L2 inner product with quadrature weight dx,
- solve the saddle-point system,
- (optional but recommended) compute the inf-sup constant β.

*Sanity checks (noise-free):* after reconstruction, `observe(u_rec, W, dx)` should match `y_obs` up to ~1e-12.


In [ ]:
@dataclass
class PBDWModel:
    """Offline quantities for PBDW"""
    Z: np.ndarray          # Background basis (Nq × N)
    Q: np.ndarray          # Riesz representers (Nq × M)
    W: np.ndarray          # Sensor densities (M × Nq)
    A: np.ndarray          # Gramian in U_M: A = (Q,Q)_V
    B: np.ndarray          # Cross Gramian:  B = (Q,Z)_V
    K: np.ndarray          # Saddle-point matrix
    beta: float            # Inf-sup constant (stability)

def pbdw_offline(Z: np.ndarray, Q: np.ndarray, W: np.ndarray, dx: float) -> PBDWModel:
    """
    TODO 4.1: Offline stage.

    Use the discrete L2 inner product (u,v)_V ≈ dx * u^T v.

    You should build:
      A = dx * Q.T @ Q          (M×M)
      B = dx * Q.T @ Z          (M×N)
      K = [[A, B], [B.T, 0]]    ((M+N)×(M+N))

    Optional: compute the inf-sup constant beta using a normalized SVD:
      beta = σ_min( A^{-1/2} B (Z^T M Z)^{-1/2} ), with M = dx*I.
    """
    raise NotImplementedError("TODO 4.1: implement pbdw_offline")

def pbdw_online(model: PBDWModel, y_obs: np.ndarray, lam: float = 0.0):
    """
    TODO 4.2: Online reconstruction.

    Noise-free case: solve K [eta_coef; z_coef] = [y_obs; 0].

    Noisy case (Tikhonov): one practical option is to modify the (1,1) block:
      A -> A + lam*A
    (i.e., add lam times the V-norm penalty on η).

    Return:
      u_rec = Q @ eta_coef + Z @ z_coef
    """
    raise NotImplementedError("TODO 4.2: implement pbdw_online")

# Build PBDW model (will work after TODOs are completed)
model = pbdw_offline(Z, Q, W, dx)

print(f"PBDW Model:")
print(f"  Background space dimension: N = {Z.shape[1]}")
print(f"  Observable space dimension: M = {Q.shape[1]}")
print(f"  Inf-sup constant: β = {model.beta:.4f}")
print(f"  Stability factor (1 + 1/β): {1 + 1/model.beta:.2f}")
print(f"  Condition number of K: {np.linalg.cond(model.K):.2e}")


## 5. Test: Noise-Free State Estimation

**Questions.**
1. Why do we add a *model error* term? What should PBDW do with it?
2. Compare the PBDW reconstruction with the best approximation in $\mathcal{Z}_N$. When is PBDW strictly better?


In [ ]:
# Generate "true" state (with model error!)
mu_true = np.array([1.3, 0.7, 1.8])  # True parameters
kappa_true = build_conductivity(mu_true)
u_true = solve_diffusion(kappa_true)

# Add model error: true state has additional unmodeled component
model_error = 0.1 * np.sin(5 * np.pi * x) * np.exp(-10*(x-0.5)**2)
u_true_with_error = u_true + model_error

# Generate noise-free observations
y_obs = observe(u_true_with_error, W, dx)

# PBDW reconstruction
u_pbdw, eta_coef, z_coef = pbdw_online(model, y_obs)

# Also compute best approximation in Z_N (projection)
z_best_coef = np.linalg.lstsq(Z, u_true_with_error, rcond=None)[0]
u_best = Z @ z_best_coef

# Errors
err_pbdw = np.linalg.norm(u_true_with_error - u_pbdw) * np.sqrt(dx)
err_best = np.linalg.norm(u_true_with_error - u_best) * np.sqrt(dx)
err_ratio = err_pbdw / err_best

print(f"Noise-free reconstruction:")
print(f"  ||u_true - u_pbdw||_V = {err_pbdw:.6e}")
print(f"  ||u_true - u_best||_V = {err_best:.6e}  (best in Z_N)")
print(f"  Ratio: {err_ratio:.2f}  (theory: ≤ {1 + 1/model.beta:.2f})")

# Visualization
fig, axes = plt.subplots(1, 3, figsize=(15, 4))

axes[0].plot(x, u_true_with_error, 'k-', linewidth=2, label='True (with model error)')
axes[0].plot(x, u_pbdw, 'r--', linewidth=2, label='PBDW')
axes[0].plot(x, u_best, 'b:', linewidth=2, label='Best in $\mathcal{Z}_N$')
axes[0].scatter(sensor_positions, np.full(M, u_true_with_error.min()), 
                c='green', marker='^', s=50, label='Sensors')
axes[0].set_xlabel('x'); axes[0].set_ylabel('u(x)')
axes[0].set_title('State Estimation')
axes[0].legend(); axes[0].grid(True, alpha=0.3)

axes[1].plot(x, np.abs(u_true_with_error - u_pbdw), 'r-', linewidth=2, label='PBDW error')
axes[1].plot(x, np.abs(u_true_with_error - u_best), 'b--', linewidth=2, label='Best approx error')
axes[1].set_xlabel('x'); axes[1].set_ylabel('|error|')
axes[1].set_title('Pointwise Errors')
axes[1].legend(); axes[1].grid(True, alpha=0.3)

# Decomposition: z + eta
z_component = Z @ z_coef
eta_component = Q @ eta_coef
axes[2].plot(x, z_component, 'b-', linewidth=2, label='$z_N$ (model)')
axes[2].plot(x, eta_component, 'g-', linewidth=2, label='$\eta_M$ (correction)')
axes[2].plot(x, model_error, 'k--', linewidth=1, label='True model error')
axes[2].set_xlabel('x'); axes[2].set_ylabel('Component')
axes[2].set_title('PBDW Decomposition')
axes[2].legend(); axes[2].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

## 6. Test: Noisy Observations

With noisy observations $y_m^{\text{obs}} = \ell_m(u^{\text{true}}) + \epsilon_m$, we use Tikhonov regularization.

**Questions.**
1. What happens when λ=0 and the data are noisy?
2. How would you choose λ as a function of the noise level δ?


In [ ]:
# Add noise to observations
noise_levels = [0.0, 1e-3, 5e-3, 1e-2, 5e-2]

results = []
for delta in noise_levels:
    # Noisy observations
    noise = delta * np.random.randn(M)
    y_noisy = y_obs + noise
    
    # Try different regularization parameters
    if delta == 0:
        lambdas = [0.0]
    else:
        lambdas = [0.0, delta**2, delta, delta*10]
    
    for lam in lambdas:
        u_rec, _, _ = pbdw_online(model, y_noisy, lam=lam)
        err = np.linalg.norm(u_true_with_error - u_rec) * np.sqrt(dx)
        results.append((delta, lam, err))

# Display results
print("PBDW with noisy observations:")
print(f"{'Noise δ':<12} {'λ':<12} {'Error ||u-u_rec||':<20}")
print("-" * 50)
for delta, lam, err in results:
    print(f"{delta:<12.1e} {lam:<12.1e} {err:<20.6e}")

In [ ]:
# Visualize effect of noise and regularization
delta = 1e-2
noise = delta * np.random.randn(M)
y_noisy = y_obs + noise

fig, axes = plt.subplots(1, 3, figsize=(15, 4))

lambdas_plot = [0.0, delta, delta*10]
titles = ['No regularization', f'λ = δ = {delta}', f'λ = 10δ = {10*delta}']

for ax, lam, title in zip(axes, lambdas_plot, titles):
    u_rec, _, _ = pbdw_online(model, y_noisy, lam=lam)
    err = np.linalg.norm(u_true_with_error - u_rec) * np.sqrt(dx)
    
    ax.plot(x, u_true_with_error, 'k-', linewidth=2, label='True')
    ax.plot(x, u_rec, 'r--', linewidth=2, label=f'PBDW (err={err:.4f})')
    ax.set_xlabel('x'); ax.set_ylabel('u(x)')
    ax.set_title(title)
    ax.legend(); ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

## 7. Overdetermined Case: $M > N$

Using more sensors than basis functions improves robustness.

**Question.** How does increasing M affect (i) β, (ii) the condition number, and (iii) the reconstruction error?


In [ ]:
# Compare different M values
M_values = [8, 12, 20, 30, 50]
delta = 1e-2

errors_vs_M = []
betas_vs_M = []

for M_test in M_values:
    # New sensor configuration
    positions = np.linspace(0.05, 0.95, M_test)
    config = SensorConfig(positions=positions, width=0.05, type='gaussian')
    W_test = build_sensor_matrix(x, config)
    Q_test = build_riesz_representers(W_test, dx)
    
    # Build PBDW model
    model_test = pbdw_offline(Z, Q_test, W_test, dx)
    betas_vs_M.append(model_test.beta)
    
    # Noisy observations
    y_test = observe(u_true_with_error, W_test, dx)
    y_noisy_test = y_test + delta * np.random.randn(M_test)
    
    # Reconstruction with optimal lambda ~ delta
    u_rec, _, _ = pbdw_online(model_test, y_noisy_test, lam=delta)
    err = np.linalg.norm(u_true_with_error - u_rec) * np.sqrt(dx)
    errors_vs_M.append(err)
    
    print(f"M={M_test:3d}: β={model_test.beta:.4f}, error={err:.6e}")

# Plot
fig, axes = plt.subplots(1, 2, figsize=(12, 4))

axes[0].plot(M_values, betas_vs_M, 'bo-', markersize=8)
axes[0].axhline(0, color='r', linestyle='--')
axes[0].set_xlabel('M (number of sensors)')
axes[0].set_ylabel('Inf-sup constant β')
axes[0].set_title(f'Stability vs Sensors (N={N})')
axes[0].grid(True, alpha=0.3)

axes[1].semilogy(M_values, errors_vs_M, 'rs-', markersize=8)
axes[1].axhline(err_best, color='gray', linestyle=':', label='Best approx error')
axes[1].set_xlabel('M (number of sensors)')
axes[1].set_ylabel('Reconstruction error')
axes[1].set_title(f'Error vs Sensors (noise δ={delta})')
axes[1].legend(); axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

## 8. GEIM + PBDW: Combined Approach

Use GEIM to select optimal sensors, then PBDW for state estimation.

**Motivation**: Uniform sensor placement is simple but may not be optimal. GEIM provides a principled way to select sensors that maximize the inf-sup constant $\beta_{N,M}$, leading to better stability and accuracy.

**TODO (implementation).** Implement GEIM greedy sensor selection in the code cell below. You may follow the algorithm:
1. Pick the snapshot with largest norm.
2. Select the sensor (row of the dictionary) with maximal absolute response.
3. Normalize to build the first GEIM basis function.
4. Iterate on the worst-approximated snapshot.

*Hint:* for this 1D demo, you may use the discrete dot product `A_dict @ v` as the sensor response.

**Questions:**
1. Why does GEIM select sensors at certain locations? How does this relate to the structure of the solution manifold?
2. Compare the inf-sup constants for uniform vs GEIM sensors. Which is larger and why?

In [ ]:
def geim_sensor_selection(S: np.ndarray, A_dict: np.ndarray, M_geim: int, tol: float = 1e-10):
    """
    TODO 8.1: GEIM greedy algorithm for sensor selection.

    Args:
        S: Snapshot matrix (Nq × n_train)
        A_dict: Dictionary of candidate sensors (Ns × Nq) (rows = sensor densities)
        M_geim: number of sensors to select

    Returns:
        J: list of selected sensor indices (length M_geim)
        Q_geim: GEIM basis matrix (Nq × M_geim)
    """
    raise NotImplementedError("TODO 8.1: implement GEIM greedy selection")

# Build sensor dictionary (candidate sensors)
Ns_dict = 50
dict_positions = np.linspace(0.02, 0.98, Ns_dict)
dict_config = SensorConfig(positions=dict_positions, width=0.04, type='gaussian')
A_dict = build_sensor_matrix(x, dict_config)

# GEIM sensor selection
M_geim = 15
J_geim, Q_geim = geim_sensor_selection(S, A_dict, M_geim)

print(f"GEIM selected sensors: {J_geim}")
print(f"Selected positions: {[f'{dict_positions[j]:.2f}' for j in J_geim]}")

# Build PBDW model with GEIM-selected sensors
W_geim = A_dict[J_geim, :]
Q_riesz_geim = build_riesz_representers(W_geim, dx)
model_geim = pbdw_offline(Z, Q_riesz_geim, W_geim, dx)

print(f"GEIM+PBDW model:")
print(f"  Inf-sup constant: β = {model_geim.beta:.4f}")
print(f"  Stability factor: {1 + 1/model_geim.beta:.2f}")


In [ ]:
# Compare uniform vs GEIM sensor selection
delta = 1e-2

# Uniform sensors (original)
y_uniform = observe(u_true_with_error, W, dx) + delta * np.random.randn(M)
u_uniform, _, _ = pbdw_online(model, y_uniform, lam=delta)
err_uniform = np.linalg.norm(u_true_with_error - u_uniform) * np.sqrt(dx)

# GEIM sensors
y_geim = observe(u_true_with_error, W_geim, dx) + delta * np.random.randn(len(J_geim))
u_geim_pbdw, _, _ = pbdw_online(model_geim, y_geim, lam=delta)
err_geim = np.linalg.norm(u_true_with_error - u_geim_pbdw) * np.sqrt(dx)

print(f"Comparison (noise δ={delta}):")
print(f"  Uniform sensors:  β={model.beta:.4f}, error={err_uniform:.6e}")
print(f"  GEIM sensors:     β={model_geim.beta:.4f}, error={err_geim:.6e}")

# Visualization
fig, axes = plt.subplots(1, 2, figsize=(14, 4))

axes[0].plot(x, u_true_with_error, 'k-', linewidth=2, label='True')
axes[0].plot(x, u_uniform, 'b--', linewidth=2, label=f'Uniform (err={err_uniform:.4f})')
axes[0].plot(x, u_geim_pbdw, 'r--', linewidth=2, label=f'GEIM (err={err_geim:.4f})')
axes[0].scatter(sensor_positions, np.zeros(M) + u_true_with_error.min()*0.9, 
                c='blue', marker='^', s=40, label='Uniform sensors')
axes[0].scatter(dict_positions[J_geim], np.zeros(len(J_geim)) + u_true_with_error.min()*0.95, 
                c='red', marker='v', s=40, label='GEIM sensors')
axes[0].set_xlabel('x'); axes[0].set_ylabel('u(x)')
axes[0].set_title('PBDW: Uniform vs GEIM Sensors')
axes[0].legend(); axes[0].grid(True, alpha=0.3)

# Error distribution
axes[1].plot(x, np.abs(u_true_with_error - u_uniform), 'b-', linewidth=2, label='Uniform')
axes[1].plot(x, np.abs(u_true_with_error - u_geim_pbdw), 'r-', linewidth=2, label='GEIM')
axes[1].set_xlabel('x'); axes[1].set_ylabel('|error|')
axes[1].set_title('Pointwise Errors')
axes[1].legend(); axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

## 9. Exercise: Systematic Comparison of GEIM+PBDW vs Uniform+PBDW

In this exercise, you will perform a comprehensive comparison between uniform sensor placement and GEIM-based optimal sensor selection within the PBDW framework.

**TODO 9.1: Convergence study with increasing M**

Study how the inf-sup constant $\beta_{N,M}$ and reconstruction error evolve as the number of sensors $M$ increases, for both uniform and GEIM sensor placement.

Complete the code cell below to:
1. For $M \in \{N, N+2, N+4, \ldots, 3N\}$, compute:
   - $\beta_{N,M}$ for uniform sensors
   - $\beta_{N,M}$ for GEIM-selected sensors
   - Reconstruction error (with noise $\delta = 10^{-2}$) for both
2. Plot $\beta_{N,M}$ vs $M$ and error vs $M$ for both strategies

**Questions:**
1. For what value of $M$ does $\beta_{N,M}$ stabilize? Is there a benefit to using $M \gg N$?
2. Does GEIM always outperform uniform placement? Under what conditions might uniform be sufficient?
3. How does the theoretical bound $(1 + 1/\beta_{N,M}) \inf_{z \in \mathcal{Z}_N} \|u^{\text{true}} - z\|_V$ compare to the actual error?

In [ ]:
# TODO 9.1: Convergence study - GEIM vs Uniform sensor placement

def convergence_study_geim_vs_uniform(Z, S, x, dx, u_true, A_dict, dict_positions, 
                                       M_values, delta=1e-2, n_trials=10):
    """
    TODO: Complete this function to compare GEIM vs uniform sensor placement.
    
    For each M in M_values:
    1. Build uniform sensors and compute beta_uniform, err_uniform
    2. Run GEIM to select M sensors and compute beta_geim, err_geim
    3. Average errors over n_trials (different noise realizations)
    
    Returns:
        results: dict with keys 'M', 'beta_uniform', 'beta_geim', 
                 'err_uniform', 'err_geim'
    """
    results = {
        'M': M_values,
        'beta_uniform': [],
        'beta_geim': [],
        'err_uniform': [],
        'err_geim': [],
        'geim_positions': []
    }
    
    for M_test in M_values:
        print(f"Processing M = {M_test}...")
        
        # --- Uniform sensors ---
        # TODO: Build uniform sensor configuration
        # positions_uniform = ...
        # config_uniform = SensorConfig(...)
        # W_uniform = build_sensor_matrix(x, config_uniform)
        # Q_uniform = build_riesz_representers(W_uniform, dx)
        # model_uniform = pbdw_offline(Z, Q_uniform, W_uniform, dx)
        
        # --- GEIM sensors ---
        # TODO: Run GEIM selection with M_test sensors
        # J_geim, Q_geim = geim_sensor_selection(S, A_dict, M_test)
        # W_geim = A_dict[J_geim, :]
        # Q_riesz_geim = build_riesz_representers(W_geim, dx)
        # model_geim = pbdw_offline(Z, Q_riesz_geim, W_geim, dx)
        
        # --- Compute errors (average over noise realizations) ---
        # TODO: For each trial, add noise and compute reconstruction error
        # err_uniform_trials = []
        # err_geim_trials = []
        # for trial in range(n_trials):
        #     noise_uniform = delta * np.random.randn(M_test)
        #     noise_geim = delta * np.random.randn(M_test)
        #     ...
        
        # TODO: Store results
        # results['beta_uniform'].append(model_uniform.beta)
        # results['beta_geim'].append(model_geim.beta)
        # results['err_uniform'].append(np.mean(err_uniform_trials))
        # results['err_geim'].append(np.mean(err_geim_trials))
        # results['geim_positions'].append(dict_positions[J_geim])
        
        raise NotImplementedError("TODO 9.1: Complete the convergence study")
    
    return results

# Run the convergence study
N = Z.shape[1]
M_values = list(range(N, 3*N + 1, 2))  # M from N to 3N

# TODO: Uncomment after implementing convergence_study_geim_vs_uniform
# results = convergence_study_geim_vs_uniform(
#     Z, S, x, dx, u_true_with_error, A_dict, dict_positions, M_values
# )

print("TODO: Implement convergence_study_geim_vs_uniform and run the study")

In [ ]:
# TODO 9.2: Visualization of convergence results
# After completing TODO 9.1, run this cell to visualize the results

def plot_convergence_results(results, err_best):
    """Plot the convergence study results."""
    fig, axes = plt.subplots(1, 3, figsize=(15, 4))
    
    # Plot 1: Inf-sup constant vs M
    axes[0].plot(results['M'], results['beta_uniform'], 'bo-', 
                 markersize=8, linewidth=2, label='Uniform')
    axes[0].plot(results['M'], results['beta_geim'], 'rs-', 
                 markersize=8, linewidth=2, label='GEIM')
    axes[0].set_xlabel('M (number of sensors)')
    axes[0].set_ylabel('Inf-sup constant β')
    axes[0].set_title('Stability: β vs Number of Sensors')
    axes[0].legend()
    axes[0].grid(True, alpha=0.3)
    
    # Plot 2: Reconstruction error vs M
    axes[1].semilogy(results['M'], results['err_uniform'], 'bo-', 
                     markersize=8, linewidth=2, label='Uniform')
    axes[1].semilogy(results['M'], results['err_geim'], 'rs-', 
                     markersize=8, linewidth=2, label='GEIM')
    axes[1].axhline(err_best, color='gray', linestyle=':', 
                    linewidth=2, label='Best approx in $\mathcal{Z}_N$')
    axes[1].set_xlabel('M (number of sensors)')
    axes[1].set_ylabel('Reconstruction error')
    axes[1].set_title('Accuracy: Error vs Number of Sensors')
    axes[1].legend()
    axes[1].grid(True, alpha=0.3)
    
    # Plot 3: Theoretical bound vs actual error
    theoretical_uniform = [(1 + 1/b) * err_best for b in results['beta_uniform']]
    theoretical_geim = [(1 + 1/b) * err_best for b in results['beta_geim']]
    
    axes[2].semilogy(results['M'], theoretical_uniform, 'b--', 
                     linewidth=2, label='Bound (Uniform)')
    axes[2].semilogy(results['M'], results['err_uniform'], 'bo', 
                     markersize=8, label='Actual (Uniform)')
    axes[2].semilogy(results['M'], theoretical_geim, 'r--', 
                     linewidth=2, label='Bound (GEIM)')
    axes[2].semilogy(results['M'], results['err_geim'], 'rs', 
                     markersize=8, label='Actual (GEIM)')
    axes[2].set_xlabel('M (number of sensors)')
    axes[2].set_ylabel('Error')
    axes[2].set_title('Theoretical Bound vs Actual Error')
    axes[2].legend()
    axes[2].grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.show()

# TODO: Uncomment after completing TODO 9.1
# plot_convergence_results(results, err_best)

print("TODO: Run after completing the convergence study")

## 10. Exercise: Noise Robustness Analysis

**TODO 9.3: Study how GEIM+PBDW and Uniform+PBDW behave under different noise levels.**

For a fixed number of sensors $M = 2N$, compare the reconstruction error as the noise level $\delta$ varies from $10^{-4}$ to $10^{-1}$.

**Questions:**
1. At what noise level does regularization become essential?
2. Does GEIM provide more robustness to noise than uniform placement?
3. How should the regularization parameter $\lambda$ scale with $\delta$?

In [ ]:
# TODO 9.3: Noise robustness study

def noise_robustness_study(model_uniform, model_geim, W_uniform, W_geim, 
                           u_true, dx, noise_levels, n_trials=20):
    """
    TODO: Compare GEIM vs Uniform under varying noise levels.
    
    For each noise level delta:
    1. Generate noisy observations
    2. Reconstruct with optimal regularization lambda ~ delta
    3. Compute and store reconstruction errors
    
    Returns:
        results: dict with keys 'delta', 'err_uniform', 'err_geim',
                 'err_uniform_no_reg', 'err_geim_no_reg'
    """
    results = {
        'delta': noise_levels,
        'err_uniform': [],
        'err_geim': [],
        'err_uniform_no_reg': [],
        'err_geim_no_reg': []
    }
    
    for delta in noise_levels:
        print(f"Processing noise level δ = {delta:.1e}...")
        
        # TODO: For each noise level, compute average error over n_trials
        # Consider both with regularization (lambda = delta) and without (lambda = 0)
        
        # err_uniform_trials = []
        # err_geim_trials = []
        # err_uniform_no_reg_trials = []
        # err_geim_no_reg_trials = []
        
        # for trial in range(n_trials):
        #     ...
        
        raise NotImplementedError("TODO 9.3: Complete the noise robustness study")
    
    return results

# Define noise levels to test
noise_levels = [1e-4, 5e-4, 1e-3, 5e-3, 1e-2, 5e-2, 1e-1]

# TODO: Build models with M = 2*N sensors for both uniform and GEIM
# M_test = 2 * N
# ... (build W_uniform_2N, model_uniform_2N, W_geim_2N, model_geim_2N)

# TODO: Run the noise robustness study
# noise_results = noise_robustness_study(...)

print("TODO: Implement and run the noise robustness study")

In [ ]:
# TODO 9.4: Visualization of noise robustness results

def plot_noise_robustness(noise_results):
    """Plot noise robustness study results."""
    fig, axes = plt.subplots(1, 2, figsize=(12, 4))
    
    # Plot 1: Error vs noise level (with regularization)
    axes[0].loglog(noise_results['delta'], noise_results['err_uniform'], 'bo-', 
                   markersize=8, linewidth=2, label='Uniform (with reg)')
    axes[0].loglog(noise_results['delta'], noise_results['err_geim'], 'rs-', 
                   markersize=8, linewidth=2, label='GEIM (with reg)')
    axes[0].set_xlabel('Noise level δ')
    axes[0].set_ylabel('Reconstruction error')
    axes[0].set_title('Error vs Noise Level (λ = δ)')
    axes[0].legend()
    axes[0].grid(True, alpha=0.3, which='both')
    
    # Plot 2: Effect of regularization
    axes[1].loglog(noise_results['delta'], noise_results['err_uniform_no_reg'], 'b--', 
                   markersize=6, linewidth=1, label='Uniform (no reg)')
    axes[1].loglog(noise_results['delta'], noise_results['err_uniform'], 'bo-', 
                   markersize=8, linewidth=2, label='Uniform (with reg)')
    axes[1].loglog(noise_results['delta'], noise_results['err_geim_no_reg'], 'r--', 
                   markersize=6, linewidth=1, label='GEIM (no reg)')
    axes[1].loglog(noise_results['delta'], noise_results['err_geim'], 'rs-', 
                   markersize=8, linewidth=2, label='GEIM (with reg)')
    axes[1].set_xlabel('Noise level δ')
    axes[1].set_ylabel('Reconstruction error')
    axes[1].set_title('Effect of Regularization')
    axes[1].legend()
    axes[1].grid(True, alpha=0.3, which='both')
    
    plt.tight_layout()
    plt.show()

# TODO: Uncomment after completing TODO 9.3
# plot_noise_robustness(noise_results)

print("TODO: Run after completing the noise robustness study")

## 11. Summary and Deliverables

### PBDW Key Points:

1. **Decomposition**: $u_{N,M} = z_N + \eta_M$
   - $z_N \in \mathcal{Z}_N$: Model-based component (anticipated physics)
   - $\eta_M \in \mathcal{U}_M$: Data-based correction (unmodeled physics)

2. **Error bound**: $\|u^{\text{true}} - u_{N,M}\|_V \leq (1 + 1/\beta_{N,M}) \inf_{z \in \mathcal{Z}_N} \|u^{\text{true}} - z\|_V$

3. **Inf-sup constant**: $\beta_{N,M} = \sqrt{\lambda_{\min}(\mathbf{B}^T \mathbf{A}^{-1} \mathbf{B})}$ (for orthonormal $\mathcal{Z}_N$)

4. **Noise robustness**: Tikhonov regularization with $\lambda \sim \delta$ (noise level)

5. **GEIM + PBDW synergy**:
   - GEIM provides optimal sensor selection maximizing $\beta_{N,M}$
   - PBDW provides rigorous state estimation with error bounds
   - Combined approach: better stability and accuracy than uniform placement

### References:
- Maday, Patera, Penn, Yano (2014): *A parameterized-background data-weak approach to variational data assimilation*
- Maday, Mula, Patera, Yano (2015): *The generalized empirical interpolation method: stability theory on Hilbert spaces with an application to the Stokes equation*
- Binev, Cohen, Dahmen, DeVore, Petrova, Wojtaszczyk (2017): *Data assimilation in reduced modeling*

### Deliverables:

**Report** (2-3 pages) summarizing your numerical findings:

1. **Implementation**: Briefly describe your implementation of POD, sensors, PBDW offline/online, and GEIM.

2. **Noise-free case** (Section 5): 
   - Compare PBDW reconstruction with best approximation in $\mathcal{Z}_N$
   - Verify the error bound $(1 + 1/\beta_{N,M})$

3. **Noisy case** (Section 6): 
   - Effect of noise on reconstruction
   - Role of regularization parameter $\lambda$

4. **Sensor count** (Section 7):
   - How does increasing $M$ affect $\beta_{N,M}$ and accuracy?

5. **GEIM vs Uniform** (Sections 9-10):
   - Convergence study: $\beta_{N,M}$ and error vs $M$
   - Noise robustness comparison
   - When is GEIM beneficial?

**Include**: Key plots and tables supporting your conclusions.

In [ ]:
# Final summary statistics (run after completing all exercises)

def print_final_summary():
    """Print comprehensive summary of PBDW study."""
    print("="*70)
    print("PBDW STATE ESTIMATION - FINAL SUMMARY")
    print("="*70)
    
    print(f"\n1. BACKGROUND SPACE (POD):")
    print(f"   Dimension N = {N}")
    print(f"   Energy captured: {energy[N-1]:.2%}")
    print(f"   Best approximation error: {err_best:.6e}")
    
    print(f"\n2. OBSERVABLE SPACE (Uniform sensors):")
    print(f"   Number of sensors M = {M}")
    print(f"   Sensor type: {sensor_config.type}")
    print(f"   Inf-sup constant β = {model.beta:.4f}")
    print(f"   Stability factor (1 + 1/β) = {1 + 1/model.beta:.2f}")
    
    print(f"\n3. NOISE-FREE RECONSTRUCTION:")
    print(f"   PBDW error: {err_pbdw:.6e}")
    print(f"   Error ratio (PBDW/best): {err_ratio:.2f}")
    print(f"   Theoretical bound: {1 + 1/model.beta:.2f}")
    
    print(f"\n4. GEIM+PBDW:")
    print(f"   Selected {len(J_geim)} sensors from {Ns_dict} candidates")
    print(f"   Inf-sup constant β = {model_geim.beta:.4f}")
    print(f"   Improvement factor: {model_geim.beta / model.beta:.2f}x")
    
    print("\n" + "="*70)
    print("KEY TAKEAWAYS:")
    print("="*70)
    print("• PBDW combines model knowledge (Z_N) with data (U_M)")
    print("• Inf-sup constant β controls stability and error amplification")
    print("• GEIM optimizes sensor placement to maximize β")
    print("• Regularization essential for noisy observations")
    print("="*70)

# Run summary (will work after all TODOs are completed)
try:
    print_final_summary()
except NameError as e:
    print(f"Complete all exercises first. Missing: {e}")